In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from collections import OrderedDict
from datetime import datetime, timedelta
import time
import math
import random

from joblib import dump, load
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix

from scipy.stats import rankdata
from tabulate import tabulate

# Get All Input Files

In [2]:
#input_path = './predict_inputs/'
input_path = './predict_inputs/'
input_files = [input_path + file for file in os.listdir(input_path) if file.endswith('DRF')]
print(input_files)

['./predict_inputs/GPX0512.DRF']


# Add Number of Entrants to Input File

In [3]:
def add_entrants(input_file):
    '''
        Find highest post position for each race and use as number of entrants
        
        Args:
            file (string): path to results file
            
        Returns:
            Nothing
    '''
    # Load files 
    input_df = pd.read_csv(input_file, header=None)
    
    # Find how many races are in each
    race_col = 2
    num_input_races = input_df[race_col].max()
    
    # For each race, count entrants, append as last column
    input_last_col = input_df.columns.max() + 1
    race_entrants = {}
    for race in range(1,num_input_races+1):
        entrants = input_df.loc[input_df[race_col] == race].shape[0]
        iloc = input_df.loc[input_df[race_col] == race].index
        input_df.loc[iloc, input_last_col] = pd.Series(entrants, index=iloc)
        race_entrants[race] = entrants
        
    # Save back to file
    input_df.to_csv(input_file, header=False, index=False)
    
for file in input_files:
    add_entrants(file)

# Parse Files for Input Features

In [4]:
input_map = OrderedDict({
        1: 'date',
        2: 'race_num',
        3: 'post_pos',
        1435: 'num_entrants',
        44: 'horse_name',
        42: 'entry',
        855: 'last_speed_rating',
        216: 'speed_par',
        33: 'app_weight_alw',
        64: 'starts_at_dist',
        96: 'lt_starts',
        100: 'lt_earnings',
        43: 'ml_odds',
        28: 'trainer_starts',
        29: 'trainer_wins',
        34: 'jockey_starts',
        35: 'jockey_wins'
    })

# Iterate through all input files
master_df = pd.DataFrame()
for file in input_files:
    # Open files to dataframe -- Take only columns that are necessary
    input_cols = [k for k in input_map.keys()]
    input_tmp = pd.read_csv(file, header=None)[input_cols]
    
    # Rename cols
    input_tmp.columns = [input_map[col] for col in input_tmp.columns]    
    
    # Get Track Code
    track_code = file[file.rfind('/')+1:][:3]
    input_tmp['track_code'] = track_code
    
    # Add these inputs/results to dataframes
    master_df = master_df.append(input_tmp)
    
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,jockey_starts,jockey_wins,last_speed_rating,speed_par,num_entrants,trainer_starts,trainer_wins,track_code
0,1,20190512,1,1,85828,NaN,18,1,6.0,SALSA JACK,441,78,79.0,86,7.0,8,1,GPX
1,6,20190512,1,2,120950,10.0,44,2,12.0,BENEFICIARY,28,3,84.0,86,7.0,19,0,GPX
2,0,20190512,1,3,57530,NaN,23,3,4.5,MCMANAMAN,48,5,78.0,86,7.0,31,2,GPX
3,1,20190512,1,4,69612,NaN,18,4,6.0,GRAN KING OF KINGS,251,22,79.0,86,7.0,92,18,GPX
4,19,20190512,1,5,211682,NaN,73,5,1.6,W W CONCERTO,20,4,80.0,86,7.0,135,35,GPX


# Derive Additional Features

In [5]:
def get_jockey_win_pct(df):
    df['jockey_win_pct'] = df['jockey_wins'] / df['jockey_starts']
    return df.drop(['jockey_wins', 'jockey_starts'], axis=1)

def get_trainer_win_pct(df):
    df['trainer_win_pct'] = df['trainer_wins'] / df['trainer_starts']
    return df.drop(['trainer_wins', 'trainer_starts'], axis=1)


# Calculate jockey/trainer win percentages
master_df = get_jockey_win_pct(master_df)
master_df = get_trainer_win_pct(master_df)

# Clean NaN cols
master_df = master_df.fillna(value=0)
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,last_speed_rating,speed_par,num_entrants,track_code,jockey_win_pct,trainer_win_pct
0,1,20190512,1,1,85828,0.0,18,1,6.0,SALSA JACK,79.0,86,7.0,GPX,0.176871,0.125000
1,6,20190512,1,2,120950,10.0,44,2,12.0,BENEFICIARY,84.0,86,7.0,GPX,0.107143,0.000000
2,0,20190512,1,3,57530,0.0,23,3,4.5,MCMANAMAN,78.0,86,7.0,GPX,0.104167,0.064516
3,1,20190512,1,4,69612,0.0,18,4,6.0,GRAN KING OF KINGS,79.0,86,7.0,GPX,0.087649,0.195652
4,19,20190512,1,5,211682,0.0,73,5,1.6,W W CONCERTO,80.0,86,7.0,GPX,0.200000,0.259259


# Get Past Performance Data

In [6]:
# Map for past performance fields
input_keys = OrderedDict({
    1: 'date',
    2: 'race_num',
    44: 'horse_name',
})

input_pp_map = OrderedDict({
    615 : 'pp_finish_pos',
    345 : 'pp_num_entrants',
    5: 'todays_distance',
    315: 'pp_distance',
    #535 : 'pp_race_class',
    255 : 'pp_race_date',
    1045: 'pp_claimed',
    1125: 'pp_favorite',
    605: 'pp_stretch_pos',
    113: 'pp_workout_time',
    465: 'pp_winners_margin'
})

def get_past_performance_data(input_files, num_races=3):
    '''
        Go through input files and get past performance data
        
        Args: 
            input_files (list): list of input file names 
            num_races (int): number of races back to grab data for
            
        Returns:
            pd.DataFrame with past performance data for each horse
    '''
    pp_df = pd.DataFrame()
    
    # Create new mapping for this number of races
    updated_pp_map = OrderedDict({k+race:'{}_{}'.format(v, race) for k,v in input_pp_map.items() for race in range(num_races)})
    
    for file in input_files:
        # Open input file to df
        df = pd.read_csv(file, header=None)
        
        # Only include key/past performance columns
        incl_cols = list(input_keys.keys()) + list(updated_pp_map.keys())
        df = df[incl_cols]
        
        # Rename columns
        renamed_cols = []
        for col in df.columns:
            try:
                # PP Column
                rn_col = updated_pp_map[col]
            except:
                # Key Column 
                rn_col = input_keys[col]    
            renamed_cols.append(rn_col)
        df.columns = renamed_cols
        
        # Append this file's df to pp_df
        pp_df = pp_df.append(df)
    
    return pp_df

num_races = 3
pp_df = get_past_performance_data(input_files, num_races)
pp_df.head()

,date,race_num,horse_name,pp_race_date_1,pp_race_date_2,todays_distance_0,todays_distance_1,todays_distance_2,pp_winners_margin_0,pp_winners_margin_1,...,pp_finish_pos_1,pp_finish_pos_2,pp_favorite_2,pp_workout_time_0,pp_workout_time_1,pp_workout_time_2,pp_distance_0,pp_distance_1,pp_distance_2,pp_race_date_0
0,20190512,1,SALSA JACK,20190324.0,20190313.0,1870,D,NaN,1.25,0.25,...,1.0,9.0,0.0,37.6,37.6,49.4,1870.0,1800.0,1760.0,20190426.0
1,20190512,1,BENEFICIARY,20190413.0,20190330.0,1870,D,NaN,1.75,4.25,...,3.0,10.0,0.0,37.8,-61.8,50.2,-1650.0,1540.0,1540.0,20190427.0
2,20190512,1,MCMANAMAN,20190321.0,20190222.0,1870,D,NaN,0.50,0.25,...,7.0,2.0,0.0,48.0,49.8,48.6,1760.0,1760.0,1760.0,20190407.0
3,20190512,1,GRAN KING OF KINGS,20190303.0,20190123.0,1870,D,NaN,1.25,0.75,...,6.0,1.0,0.0,51.4,62.2,49.0,1870.0,1870.0,1760.0,20190426.0
4,20190512,1,W W CONCERTO,20190331.0,20190317.0,1870,D,NaN,0.25,1.50,...,2.0,1.0,1.0,47.6,49.2,52.0,1760.0,1870.0,1760.0,20190413.0


# Clean Past Performance Data

In [7]:
def clean_finish_pos_cols(pp_df, num_races=3):
        # Get finish position column names
        finish_pos_cols = ['pp_finish_pos_{}'.format(race) for race in range(num_races)]
        
        def clean_val(val):
            # If value is numeric, return its integer rep
            try:
                return int(val)
            except:
                # If value not numeric, return 0 (did not finish)
                return int(92)
            
        # Clean columns
        for col in finish_pos_cols:
            pp_df[col] = pp_df[col].apply(clean_val)
            
        return pp_df
    
def clean_stretch_pos_cols(pp_df, num_races=3):
        # Get stretch position column names
        stretch_pos_cols = ['pp_stretch_pos_{}'.format(race) for race in range(num_races)]
        
        def clean_val(val):
            # If value is numeric, return its integer rep
            try:
                return int(val)
            except:
                # If value not numeric, return 92 (did not finish)
                return int(92)
            
        
        # Clean columns
        for col in stretch_pos_cols:
            pp_df[col] = pp_df[col].apply(clean_val)
            
        return pp_df
    
def clean_favorite_cols(pp_df, num_races=3):
    def conv(val):
        if math.isnan(val):
            return int(0)
        else:
            return int(val)
        
    # Get favorite column names
    favorite_columns = ['pp_favorite_{}'.format(race) for race in range(num_races)]
    
    # Convert to integers
    for col in favorite_columns:
        pp_df[col] = pp_df[col].apply(conv)
        
    return pp_df
    
    
def clean_pp_df(pp_df, num_races=3):
    '''
        Clean fields in pp_df
        
        Args:
            pp_df (pd.DataFrame): dataframe to clean
            
        Returns:
            (pd.DataFrame) cleaned version of pp_df
    '''
    pp_df = clean_finish_pos_cols(pp_df, num_races)
    pp_df = clean_stretch_pos_cols(pp_df, num_races)
    pp_df = clean_favorite_cols(pp_df, num_races)
    
    return pp_df

pp_df = clean_pp_df(pp_df, num_races)
pp_df.head(10)

,date,race_num,horse_name,pp_race_date_1,pp_race_date_2,todays_distance_0,todays_distance_1,todays_distance_2,pp_winners_margin_0,pp_winners_margin_1,...,pp_finish_pos_1,pp_finish_pos_2,pp_favorite_2,pp_workout_time_0,pp_workout_time_1,pp_workout_time_2,pp_distance_0,pp_distance_1,pp_distance_2,pp_race_date_0
0,20190512,1,SALSA JACK,20190324.0,20190313.0,1870,D,NaN,1.25,0.25,...,1,9,0,37.6,37.6,49.4,1870.0,1800.0,1760.0,20190426.0
1,20190512,1,BENEFICIARY,20190413.0,20190330.0,1870,D,NaN,1.75,4.25,...,3,10,0,37.8,-61.8,50.2,-1650.0,1540.0,1540.0,20190427.0
2,20190512,1,MCMANAMAN,20190321.0,20190222.0,1870,D,NaN,0.50,0.25,...,7,2,0,48.0,49.8,48.6,1760.0,1760.0,1760.0,20190407.0
3,20190512,1,GRAN KING OF KINGS,20190303.0,20190123.0,1870,D,NaN,1.25,0.75,...,6,1,0,51.4,62.2,49.0,1870.0,1870.0,1760.0,20190426.0
4,20190512,1,W W CONCERTO,20190331.0,20190317.0,1870,D,NaN,0.25,1.50,...,2,1,1,47.6,49.2,52.0,1760.0,1870.0,1760.0,20190413.0
5,20190512,1,PROUD ENOUGH,20190113.0,20181229.0,1870,D,NaN,0.75,3.50,...,6,5,0,49.6,49.0,50.4,1870.0,1760.0,1540.0,20190127.0
6,20190512,1,CHIRPING,20190320.0,20190222.0,1870,D,NaN,2.00,4.25,...,8,5,0,49.4,48.8,62.8,1760.0,1430.0,1320.0,20190420.0
7,20190512,2,SOMMER LEGACY,20190413.0,20190324.0,1870,T,NaN,5.75,1.25,...,6,6,0,51.6,37.0,64.0,-1650.0,1870.0,1760.0,20190427.0
8,20190512,2,SIMMARD'S ROAR,20180618.0,NaN,1870,T,NaN,0.25,0.50,...,6,92,0,117.0,66.0,81.0,1760.0,1830.0,NaN,20180715.0
9,20190512,2,RESTART,20190314.0,20190303.0,1870,T,NaN,0.13,5.50,...,8,6,0,62.2,50.2,51.2,1760.0,1760.0,1760.0,20190420.0


# Derive Metrics with Cleaned Data

In [8]:
def get_improved_finish_pos(pp_df, num_races=3):
    '''
        Calculate if horse is improving its finish position over the past races
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            
        Returns:
            (pd.DataFrame) an updated version of pp_df with a column for improved
            finish position
    '''
    
    def all_seq_vals_less_than_one(seq):
        ''' Returns true if all values in the passed sequence are <=1 '''
        for elt in seq:
            if elt > 1:
                return False
        return True

    # Get past finish pos cols
    finish_pos_cols = ['pp_finish_pos_{}'.format(race) for race in range(num_races)]
    # Concat all finish positions for each horse in a sequential list starting with most
    # recent race 
    finish_pos_seqs = pp_df[finish_pos_cols].values.tolist()
    
    imp_finish_pos = []
    for seq in finish_pos_seqs:
        # Skip any sequence where NaN appears
        if np.nan in seq:
            imp_finish_pos.append(False)
        # Skip any seq where horse didn't finish (fractional finish pos > 1)
        elif not all_seq_vals_less_than_one(seq):
            imp_finish_pos.append(False)
        else:
            # Determine if last finish was better than previous average finish
            most_recent = seq[0]
            avg_prev_finish = np.mean(seq[1:])
            imp_finish_pos.append(most_recent < avg_prev_finish)
            
    # Assign improved finish position series to pp_df
    pp_df['imp_finish_pos'] = pd.Series(imp_finish_pos)
    
    return pp_df

def get_recent_race(pp_df, threshold=21):
    '''
        Determine if a horse has had a previous race within threshold days
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            threshold (int): days to look back
            
        Returns:
            (pd.DataFrame) updated pp_df containing bool column stating whether
            each horse had a recent race or not
    '''
    
    def conv_date(val):
        # Set NaNs to 1970
        if math.isnan(float(val)):
            val = 19700101
        val = str(int(val))
        return datetime.strptime(val, '%Y%m%d')
    
    # Determine if race most recent race is within threshold
    pp_df['today'] = pp_df['date'].apply(conv_date)
    pp_df['last_race'] = pp_df['pp_race_date_0'].apply(conv_date)
    pp_df['timedelta'] = pd.Series(timedelta(days=threshold), index=pp_df.index)
    pp_df['cutoff'] = pp_df['today'] - pp_df['timedelta']
    pp_df['recent_race'] = pp_df['cutoff'] <= pp_df['last_race'] # Cutoff happened before last race
    
    # Drop intermediate columns
    drop_cols = ['today', 'last_race', 'timedelta', 'cutoff']
    return pp_df.drop(drop_cols, axis=1)
    
def get_past_finish_pos(pp_df, num_races=3):
    '''
        Calculate previous finish positions as finish_pos/num_entrants
        
        Args:
            pp_df (pd.DataFrame) past performance dataframe
            num_races (int): number of races for which we have past performance
        
        Returns:
            (pd.DataFrame) updated pp_df with finish positions calculated
    '''
    # Get finish position/entrants columns names
    finish_pos_cols = ['pp_finish_pos_{}'.format(race) for race in range(num_races)]
    entrants_cols = ['pp_num_entrants_{}'.format(race) for race in range(num_races)]
    cols = list(zip(finish_pos_cols, entrants_cols))
    
    # Divide finish pos by num entrants, save a raw copy of finish position
    for f_col, e_col in cols:
        pp_df['raw_'+f_col] = pp_df[f_col]
        pp_df[f_col] = pp_df[f_col] / pp_df[e_col]
    
    return pp_df

def get_claimed_in_past(pp_df, num_races=3):
    '''
        Determine if a horse was claimed in the past num_races
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            
        Returns:
            (pd.DataFrame) updated version of pp_df
    '''
    # Claimed column names
    claimed_cols = ['pp_claimed_{}'.format(race) for race in range(num_races)]
    claimed_seqs = pp_df[claimed_cols].values.tolist()
    
    # Determine if claimed in recent races    
    claimed = [True if 'c' in seq else False for seq in claimed_seqs]
    
    # Assign to pp_df
    pp_df['was_claimed'] = pd.Series(claimed, index=pp_df.index).fillna(False)
    
    return pp_df

def was_favorite(pp_df, num_races=3):
    '''
        Determine if a horse was the favorite in any of its recent races
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
    '''
    # Get favorite columns
    favorite_cols = ['pp_favorite_{}'.format(race) for race in range(num_races)]
    favorite_seqs = pp_df[favorite_cols].values.tolist()
    
    # If 1 in sequence, horse was favorite in recent race
    was_favorite = [True if 1 in seq else False for seq in favorite_seqs]
    
    pp_df['was_favorite'] = pd.Series(was_favorite, index=pp_df.index)
    
    return pp_df

def improved_stretch_pos(pp_df, num_races=3):
    '''
        Determine if a horse improved its finish position down the stretch in the most
        recent race
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
    '''
    # Stretch position columns
    stretch_pos_cols = ['pp_stretch_pos_{}'.format(race) for race in range(num_races)]
    
    # Get fractional stretch position
    pp_df['pp_stretch_pos'] = pp_df['pp_stretch_pos_0'] / pp_df['pp_num_entrants_0']
    
    # Determine if stretch pos > finish pos
    pp_df['improved_stretch_pos'] = pp_df['pp_stretch_pos'] > pp_df['pp_finish_pos_0']
    
    return pp_df.drop(['pp_stretch_pos'], axis=1)

def had_bullet_workout(pp_df, num_races=3):
    '''
        Determine if a horse had a bullet workout in previous workout
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
    '''
    # Get workout time columns
    workout_cols = ['pp_workout_time_{}'.format(race) for race in range(num_races)]
    
    # Determine if last workout was bullet
    pp_df['had_bullet'] = pp_df['pp_workout_time_0'] < 0
    
    return pp_df

def won_by_margin(pp_df, num_races=3, margin=5):
    '''
        Determine if a horse won last race by at least margin lengths
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            margin (float): number of lengths by which horse had to win
            
        Returns:
            (pd.DataFrame) updated version of pp_df
    '''
    # Find all winners of last race
    winners = pp_df.loc[pp_df.raw_pp_finish_pos_0 == 1]
    
    # Find all winners with at least margin length victory
    big_winners = winners.loc[winners.pp_winners_margin_0 >= margin].index
    
    # Create new col, default to false. Set True for all horses in big_winners index
    pp_df['won_by_margin'] = pd.Series(False, index=pp_df.index)
    pp_df.loc[big_winners, 'won_by_margin'] = True
    
    return pp_df

def won_at_similar_distance(pp_df, num_races=3):
    '''
        Determine if a horse won a recent race at the same distance as today's race
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            
        Returns:
            (pd.DataFrame) updated version of pp_df
    '''
    # Get distance and finish pos cols
    distance_cols = ['pp_distance_{}'.format(race) for race in range(num_races)]
    raw_finish_cols = ['raw_pp_finish_pos_{}'.format(race) for race in range(num_races)]
    
    # Find winners at similar distance by iterating through each recent race
    winners = []
    for race in range(num_races):
        dist_col = 'pp_distance_{}'.format(race)
        finish_col = 'raw_pp_finish_pos_{}'.format(race)
        today_dist_col = 'todays_distance_0'
        
        winners += pp_df.loc[(pp_df[dist_col] == pp_df[today_dist_col]) & (pp_df[finish_col] == 1)].index.tolist()
        
    # Create new column to denote if won at similar distance -- default False, True for 
    # horses in winners
    pp_df['won_at_similar_dist'] = pd.Series(False, index=pp_df.index)
    pp_df.loc[winners, 'won_at_similar_dist'] = True

    return pp_df
    
def derive_pp_metrics(pp_df, num_races=3):
    # Past Finish Positions
    pp_df = get_past_finish_pos(pp_df, num_races)
    # Improved Finish Position
    pp_df = get_improved_finish_pos(pp_df, num_races)
    # Recent race
    pp_df = get_recent_race(pp_df, 21)
    # Claimed in Past
    pp_df = get_claimed_in_past(pp_df, num_races)
    # Favorite in past
    pp_df = was_favorite(pp_df, num_races)
    # Improved stretch pos
    pp_df = improved_stretch_pos(pp_df, num_races)
    # Workout Rating
    pp_df = had_bullet_workout(pp_df, num_races)
    # Won last race by 5+ lengths
    pp_df = won_by_margin(pp_df, num_races, margin=4)
    # Won at similar distance
    pp_df = won_at_similar_distance(pp_df, num_races)    
    
    return pp_df
    
pp_df = derive_pp_metrics(pp_df)
pp_df.head()

,date,race_num,horse_name,pp_race_date_1,pp_race_date_2,todays_distance_0,todays_distance_1,todays_distance_2,pp_winners_margin_0,pp_winners_margin_1,...,raw_pp_finish_pos_1,raw_pp_finish_pos_2,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,20190512,1,SALSA JACK,20190324.0,20190313.0,1870,D,NaN,1.25,0.25,...,1,9,False,True,False,True,True,False,False,False
1,20190512,1,BENEFICIARY,20190413.0,20190330.0,1870,D,NaN,1.75,4.25,...,3,10,False,True,False,False,False,False,False,False
2,20190512,1,MCMANAMAN,20190321.0,20190222.0,1870,D,NaN,0.50,0.25,...,7,2,True,False,True,False,True,False,False,False
3,20190512,1,GRAN KING OF KINGS,20190303.0,20190123.0,1870,D,NaN,1.25,0.75,...,6,1,False,True,False,False,False,False,False,False
4,20190512,1,W W CONCERTO,20190331.0,20190317.0,1870,D,NaN,0.25,1.50,...,2,1,True,False,True,True,False,False,False,False


# Clean Fields used in Derivations

In [9]:
def clean_deriv_columns(pp_df, num_races=3):
    '''
        Drop all columns used as intermediates during feature derivation from pp_df
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            
        Returns:
            (pd.DataFrame): a cleaned version of pp_df
    '''
    column_dict = {
        'claimed_cols' : ['pp_claimed_{}'.format(race) for race in range(num_races)],
        'date_cols' : ['pp_race_date_{}'.format(race) for race in range(num_races)],
        'distance_cols' : ['pp_distance_{}'.format(race) for race in range(num_races)],
        'entrants_cols' : ['pp_num_entrants_{}'.format(race) for race in range(num_races)],   
        'favorite_cols' : ['pp_favorite_{}'.format(race) for race in range(num_races)],
        'finish_pos_cols' : ['pp_finish_pos_{}'.format(race) for race in range(num_races)],
        'margin_cols' : ['pp_winners_margin_{}'.format(race) for race in range(num_races)],
        'raw_finish_cols' : ['raw_pp_finish_pos_{}'.format(race) for race in range(num_races)],
        'stretch_pos_cols' : ['pp_stretch_pos_{}'.format(race) for race in range(num_races)],
        'todays_distance_cols' : ['todays_distance_{}'.format(race) for race in range(num_races)],
        'workout_cols' : ['pp_workout_time_{}'.format(race) for race in range(num_races)],
    }
    
    for k, cols in column_dict.items():
        pp_df = pp_df.drop(cols, axis=1)
        
    return pp_df

pp_df = clean_deriv_columns(pp_df, num_races)
pp_df.head()

,date,race_num,horse_name,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,20190512,1,SALSA JACK,False,True,False,True,True,False,False,False
1,20190512,1,BENEFICIARY,False,True,False,False,False,False,False,False
2,20190512,1,MCMANAMAN,True,False,True,False,True,False,False,False
3,20190512,1,GRAN KING OF KINGS,False,True,False,False,False,False,False,False
4,20190512,1,W W CONCERTO,True,False,True,True,False,False,False,False


# Merge Past Performance with Input Features

In [10]:
master_df = pd.merge(master_df, pp_df,
                     how='left', left_on=['date', 'race_num', 'horse_name'],
                     right_on=['date','race_num', 'horse_name'])
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,...,jockey_win_pct,trainer_win_pct,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,1,20190512,1,1,85828,0.0,18,1,6.0,SALSA JACK,...,0.176871,0.125000,False,True,False,True,True,False,False,False
1,6,20190512,1,2,120950,10.0,44,2,12.0,BENEFICIARY,...,0.107143,0.000000,False,True,False,False,False,False,False,False
2,0,20190512,1,3,57530,0.0,23,3,4.5,MCMANAMAN,...,0.104167,0.064516,True,False,True,False,True,False,False,False
3,1,20190512,1,4,69612,0.0,18,4,6.0,GRAN KING OF KINGS,...,0.087649,0.195652,False,True,False,False,False,False,False,False
4,19,20190512,1,5,211682,0.0,73,5,1.6,W W CONCERTO,...,0.200000,0.259259,True,False,True,True,False,False,False,False


# Convert Boolean Columns to Binary

In [11]:
bool_cols = ['imp_finish_pos', 'recent_race', 'was_claimed','was_favorite', \
             'improved_stretch_pos', 'had_bullet', 'won_by_margin', \
             'won_at_similar_dist']

for col in bool_cols:
    master_df[col] = master_df[col].apply(lambda x: int(x))

master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,...,jockey_win_pct,trainer_win_pct,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,1,20190512,1,1,85828,0.0,18,1,6.0,SALSA JACK,...,0.176871,0.125000,0,1,0,1,1,0,0,0
1,6,20190512,1,2,120950,10.0,44,2,12.0,BENEFICIARY,...,0.107143,0.000000,0,1,0,0,0,0,0,0
2,0,20190512,1,3,57530,0.0,23,3,4.5,MCMANAMAN,...,0.104167,0.064516,1,0,1,0,1,0,0,0
3,1,20190512,1,4,69612,0.0,18,4,6.0,GRAN KING OF KINGS,...,0.087649,0.195652,0,1,0,0,0,0,0,0
4,19,20190512,1,5,211682,0.0,73,5,1.6,W W CONCERTO,...,0.200000,0.259259,1,0,1,1,0,0,0,0


# Standardize Data

In [12]:
def standardize_series(s):
    return (s - s.mean()) / s.std()

# Standardize all columns except explicit columns
no_std = ['race_num', 'date', 'horse_name', 'track_code', 'odds', 'top3', 't3_conf', 'win_conf', 'win', 'entry', 'h2h']
master_df['odds'] = master_df['ml_odds']
master_df = master_df.drop(['ml_odds'], axis=1)
std_cols = [col for col in master_df.columns if col not in no_std]

for col in std_cols:
    master_df[col] = standardize_series(master_df[col]).fillna(0)

# Load Models from Disk

In [13]:
itm_file = './models/itm_gbc.joblib'
win_file = './models/win_gbc.joblib'
h2h_file = './models/h2h_gbc.joblib'

itm_model = load(itm_file)
win_model = load(win_file)
h2h_model = load(h2h_file)

In [14]:
# DO ITM and WIN predictions
for idx, row in master_df.iterrows():
    print('Entry: {}/{}'.format(idx+1, master_df.shape[0]), end='\r', flush=True)
    
    # First predict if the horse will be in top 3
    top3_pred = itm_model.predict_proba(row[std_cols].values.reshape(1,-1))
    master_df.loc[idx, 'top3'] = np.argmax(top3_pred)
    master_df.loc[idx, 't3_conf'] = np.max(top3_pred)
    
    # Predict if winner
    win_pred = win_model.predict_proba(row[std_cols].values.reshape(1,-1))
    master_df.loc[idx, 'win'] = np.argmax(win_pred)
    master_df.loc[idx, 'win_conf'] = np.max(win_pred)
    
master_df.head(20)

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,horse_name,last_speed_rating,...,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist,odds,top3,t3_conf,win,win_conf
0,-0.447753,20190512,1,-1.375514,1.539294,-0.277445,0.573247,1,SALSA JACK,0.448056,...,2.015674,1.643964,-0.231921,-0.144345,-0.355793,6.0,1.0,0.614487,1.0,0.629645
1,0.980932,20190512,1,-1.047688,2.492650,3.567155,2.767070,2,BENEFICIARY,0.667781,...,-0.490997,-0.602015,-0.231921,-0.144345,-0.355793,12.0,0.0,0.547250,0.0,0.515264
2,-0.733489,20190512,1,-0.719863,0.771170,-0.277445,0.995136,3,MCMANAMAN,0.404112,...,-0.490997,1.643964,-0.231921,-0.144345,-0.355793,4.5,0.0,0.518647,0.0,0.560935
3,-0.447753,20190512,1,-0.392038,1.099125,-0.277445,0.573247,4,GRAN KING OF KINGS,0.448056,...,-0.490997,-0.602015,-0.231921,-0.144345,-0.355793,6.0,1.0,0.579847,1.0,0.565349
4,4.695510,20190512,1,-0.064213,4.955490,-0.277445,5.214027,5,W W CONCERTO,0.492001,...,2.015674,-0.602015,-0.231921,-0.144345,-0.355793,1.6,1.0,0.519663,1.0,0.631990
5,-0.162016,20190512,1,0.263612,0.652822,-0.277445,1.585781,6,PROUD ENOUGH,-0.123227,...,-0.490997,-0.602015,-0.231921,-0.144345,-0.355793,30.0,0.0,0.533717,0.0,0.577821
6,-0.447753,20190512,1,0.591437,0.762267,-0.277445,0.151358,7,CHIRPING,0.228332,...,-0.490997,1.643964,-0.231921,-0.144345,-0.355793,2.5,1.0,0.551408,1.0,0.585216
7,-0.447753,20190512,2,-1.375514,-0.738857,-0.277445,-0.523664,1,SOMMER LEGACY,0.667781,...,-0.490997,1.643964,-0.231921,-0.144345,-0.355793,20.0,0.0,0.635123,0.0,0.626797
8,-0.447753,20190512,2,-1.047688,-0.782776,-0.277445,-0.776798,2,SIMMARD'S ROAR,0.448056,...,-0.490997,-0.602015,-0.231921,-0.144345,-0.355793,6.0,0.0,0.602444,0.0,0.614877
9,-0.447753,20190512,2,-0.719863,-0.649825,-0.277445,-0.354909,3,RESTART,0.096497,...,-0.490997,-0.602015,-0.231921,-0.144345,-0.355793,3.0,1.0,0.506870,0.0,0.600621


# Get Horse-to-Horse Comparisons

In [15]:
def find_horse_index(date, race_num, track_code, post_pos):
    return master_df.loc[(master_df.date == date) &
                         (master_df.race_num == race_num) &
                         (master_df.track_code == track_code) &
                         (master_df.post_pos == post_pos)].index[0]


# Get all races
unique_races = master_df.groupby(['date', 'race_num', 'track_code']).size().reset_index()
# Add h2h columns to dataframe
master_df['h2h_count'] = pd.Series(0, index=master_df.index)
master_df['h2h'] = pd.Series('', index=master_df.index)

# Iterate through each race
for ii, row in unique_races.iterrows():
    print('Race: {}/{}'.format(ii+1, len(unique_races)), end='\r', flush=True)
    # Get race characteristics
    date = row.date
    track_code = row.track_code
    race_num = row.race_num
    # Get master_df slice for this race
    race_df = master_df.loc[(master_df.date == date) &
                           (master_df.race_num == race_num) &
                           (master_df.track_code == track_code)]
    # Only want to do comparisions between horses once. So we will loop through
    # the first half (ceiling) of post positions, and will compare to all other
    # post positions within that loop. This will result in each combination of 
    # post positions only being evaluated once.
    post_positions = sorted(race_df.post_pos)
    for ii, h1_pos in enumerate(post_positions):
        # Iterate through all other post positions
        for h2_pos in post_positions:
            # Don't compare to self or any positions that have already been evaluated
            if h1_pos <= h2_pos:
                continue
                
            # Get horses' data and merge
            h1 = race_df.loc[race_df.post_pos == h1_pos]
            h2 = race_df.loc[race_df.post_pos == h2_pos]
            
            h1_merge = h1.merge(h2, left_on=['date', 'race_num', 'track_code'],
                                              right_on=['date', 'race_num', 'track_code'])
            h2_merge = h2.merge(h1, left_on=['date', 'race_num', 'track_code'],
                                              right_on=['date', 'race_num', 'track_code'])
            
            # Drop columns not used by h2h model
            cols_to_drop = set(['date', 'race_num', 'track_code', 'horse_name_x', 'horse_name_y', 'num_entrants_y',\
                                'odds_x', 'odds_y', 'entry_x', 'entry_y','h2h', 'win_x', 'win_y', 'top3_x', 'top3_y',\
                                't3_conf_x', 't3_conf_y', 'win_conf_x', 'win_conf_y', 'h2h_count_x', 'h2h_count_y',\
                                'h2h_x', 'h2h_y'])
            cols = list(set(h1_merge.columns) - cols_to_drop)
            h1_data = h1_merge[cols]
            h2_data = h2_merge[cols]
            
            # Run horses through model
            h1_h2h = h2h_model.predict_proba(h1_data)[0]
            h2_h2h = h2h_model.predict_proba(h2_data)[0]
            
            # Get prediction and confidence level for each horse
            h1_pred = np.argmax(h1_h2h)
            h1_conf = h1_h2h[h1_pred]
            h2_pred = np.argmax(h2_h2h)
            h2_conf = h2_h2h[h2_pred]
            
            # If labels are different, then increment count for each horse based
            # solely on label
            h1_idx = find_horse_index(date, race_num, track_code, h1_pos)
            h2_idx = find_horse_index(date, race_num, track_code, h2_pos)
            if h1_pred != h2_pred:
                master_df.loc[[h1_idx], 'h2h_count'] += h1_pred
                master_df.loc[[h2_idx], 'h2h_count'] += h2_pred
            else:
                # Give label priority to horse with higher prediction confidence
                if h1_conf >= h2_conf:
                    master_df.loc[[h1_idx], 'h2h_count'] += h1_pred
                    master_df.loc[[h2_idx], 'h2h_count'] += 1 if h2_pred == 0 else 0
                else:
                    master_df.loc[[h1_idx], 'h2h_count'] += 1 if h1_pred == 0 else 0
                    master_df.loc[[h2_idx], 'h2h_count'] += h2_pred
                    
    # Turn counts into fractions
    master_df.loc[race_df.index, 'h2h'] = \
                        master_df.loc[race_df.index]['h2h_count'].apply(lambda x: '{}/{}'.format(x, len(race_df)-1))
        
master_df = master_df.drop(['h2h_count'], axis=1)
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,horse_name,last_speed_rating,...,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist,odds,top3,t3_conf,win,win_conf,h2h
0,-0.447753,20190512,1,-1.375514,1.539294,-0.277445,0.573247,1,SALSA JACK,0.448056,...,1.643964,-0.231921,-0.144345,-0.355793,6.0,1.0,0.614487,1.0,0.629645,3/6
1,0.980932,20190512,1,-1.047688,2.492650,3.567155,2.767070,2,BENEFICIARY,0.667781,...,-0.602015,-0.231921,-0.144345,-0.355793,12.0,0.0,0.547250,0.0,0.515264,1/6
2,-0.733489,20190512,1,-0.719863,0.771170,-0.277445,0.995136,3,MCMANAMAN,0.404112,...,1.643964,-0.231921,-0.144345,-0.355793,4.5,0.0,0.518647,0.0,0.560935,4/6
3,-0.447753,20190512,1,-0.392038,1.099125,-0.277445,0.573247,4,GRAN KING OF KINGS,0.448056,...,-0.602015,-0.231921,-0.144345,-0.355793,6.0,1.0,0.579847,1.0,0.565349,5/6
4,4.695510,20190512,1,-0.064213,4.955490,-0.277445,5.214027,5,W W CONCERTO,0.492001,...,-0.602015,-0.231921,-0.144345,-0.355793,1.6,1.0,0.519663,1.0,0.631990,2/6


# Print Predictions

In [16]:
# Run through day/track combination
track_date_combos = master_df.groupby(['date','track_code']).size().reset_index() # df

for idx, row in track_date_combos.iterrows():
    # Create a separate table for each race
    track_code = row['track_code']
    date = row['date']
    track_df = master_df.loc[(master_df.date == date) & (master_df.track_code == track_code)]
    cols_to_keep = ['horse_name', 'entry', 'win', 'win_conf','h2h', 'odds']#, 'top3', 't3_conf']
    out = ''
    for num in range(1, track_df.race_num.max()+1):
        # Get data for this race
        race_df = track_df.loc[track_df['race_num'] == num][cols_to_keep]
        race_df['ml_rank'] = rankdata(race_df['odds'], method='min')
        # Get predicted winners first and sort by confidence level
        win_df = race_df.loc[race_df.win == 1]
        win_df = win_df.sort_values(['win_conf'], ascending=False)
        # Get not predicted winners and sort by h2h
        h2h_df = race_df.loc[race_df.win == 0]
        h2h_df = h2h_df.sort_values('h2h', ascending=False)
        
        race_df = win_df.append(h2h_df)       
        
        out += 'RACE: {}'.format(num)
        out += '\n'
        out += tabulate(race_df, headers='keys', tablefmt='psql', showindex=False)
        out += '\n\n'

    filename= './predictions/{}{}_predictions.txt'.format(track_code, date)    
    output_file = open(filename, 'w+')
    output_file.write(out)
    output_file.close()

# Save Predictions to CSV
Save to csv to do betting analysis

In [17]:
#cols_to_keep = ['date', 'track_code', 'race_num', 'horse_name', 'entry', 'win', 'win_conf','h2h', 'odds']
#preds_df = master_df[cols_to_keep]

#filename = './predictions/predictions.csv'
#preds_df.to_csv(filename, index=False)

In [18]:
#df = pd.read_csv(filename)
#df.head()